In [1]:
import numpy as np
import pandas as pd
import os, random
from tqdm import tqdm # 진행도 시각화를 위한 라이브러리

seed=42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)

## 데이터 불러오기

In [2]:
train_data = pd.read_csv("nsmc_train.csv", index_col=0)
test_data = pd.read_csv("nsmc_test.csv", index_col=0)
print(train_data.shape)
print(test_data.shape)

(149993, 2)
(49999, 1)


In [3]:
train_data.head()

,review,rating
id,,
9324809,배우들의 인생연기가 돋보였던... 최고의 드라마,1
9305425,아 혜리 보고싶다 ... 여군좀 ㅠ,0
5239110,"눈이 팅팅..... 정말 ,..... 대박이다......",1
9148159,캐슬린 터너의 보디는 볼만했다,0
6144938,진짜 최고였다.,1


In [4]:
x_train = train_data["review"]
y_train = np.array(train_data["rating"])

## 전처리


- 불용어 제거

In [5]:
import re

pattern = r"[^\uAC00-\uD7A3a-zA-Z\s]"

def apply_regex(pattern, text):  # 정규표현식을 이용한 필터링 적용
    text = re.sub(pattern, "", text)  # 정규표현식 패턴에 맞는 값들을 텍스트에서 제거
    text = text.upper() # 영어들을 찾아 대문자로 치환하는 코드 작성
    return text

x_train_preprocessed = [apply_regex(pattern, str(x[1])) for x in tqdm(x_train.iteritems(), total=len(x_train), desc="pre-processing data")]

C:\Users\VISLAB-seongwon\AppData\Local\Temp\ipykernel_4196\3170241328.py:10: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  x_train_preprocessed = [apply_regex(pattern, str(x[1])) for x in tqdm(x_train.iteritems(), total=len(x_train), desc="pre-processing data")]
pre-processing data: 100%|██████████| 149993/149993 [00:00<00:00, 372064.39it/s]


- 토큰화

In [6]:
from konlpy.tag import Okt
okt = Okt()

def tokenize_words(sentence):
    sentence_n = okt.normalize(sentence)
    sentence_tokenized = okt.morphs(sentence_n, stem=True)
    #sentence_tokenized = okt.morphs(sentence_n, norm=True, stem=True)
    return sentence_tokenized

In [7]:
# 약 10-15분 정도 소요됩니다. 
x_train_tokenized = [tokenize_words(x) for x in tqdm(x_train_preprocessed, desc="tokenizing data")]

tokenizing data: 100%|█████████▉| 149413/149993 [06:36<00:01, 330.47it/s]

- 불용어 제거

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']  #별다른 의미가 없는 불용어들

def exclude_stopwords(text):
    # 위 리스트에 포함된 불용어들을 제거하는 코드 작성
    text_n = [word for word in text if word not in stopwords]
    return text_n

x_train_stopwords_excluded = [exclude_stopwords(x) for x in x_train_tokenized]

In [ ]:
len(x_train_stopwords_excluded)

- 단어 임베딩

In [ ]:
embedding_dict = dict()  # 단어 임베딩을 위한 딕셔너리 
embedding_value = 0

def embed_tokens(sentence_tokenized, mode):
    assert mode.upper() in ["TRAIN", "TEST"]
    global embedding_value
    
    sentence_embedded = list()
    for word in sentence_tokenized:
        # 코드 작성
        if word in sentence_tokenized:
            if word not in embedding_dict:
                embedding_dict[word] = embedding_value
                embedding_value += 1
        sentence_embedded.append(embedding_dict[word])
    
    return sentence_embedded

In [ ]:
x_train_embedded = [embed_tokens(x, mode="TRAIN") for x in tqdm(x_train_stopwords_excluded, desc="embedding data")]
print(x_train_embedded[:5])
print("총 %d개의 단어가 임베딩되었습니다."%(embedding_value))

- 문장 벡터화

In [ ]:
#########################################################################################
# Empty Module #5
# 입력: 임베딩 과정을 거친 데이터
# 출력: BoW 형태로 변환되어, M차원의 고정된 크기를 가진 벡터로 변환된 데이터
# 힌트: np.zeros((2, 3))는 [2, 3] 크기의 0으로 가득 찬 행렬을 생성합니다.
##########################################################################################
M = embedding_value # 전체 단어의 수

def to_BoW_representation(x):
    shape = (len(x), M) # BoW는 어떤 shape를 가져야 할까요?
    x_BoW = np.zeros(shape)
    for i in tqdm(range(len(x)), desc="making BoW representation"):
        # 여기에 BoW 구현
        for word_index in x[i]:
            x_BoW[i][word_index] += 1
    return x_BoW

x_train_BoW = to_BoW_representation(x_train_embedded)

- 차원축소

In [ ]:
# 여기에 코드 작성
def reduce_dimension(words):
    words_cnt = np.sum(words, axis=0) # 각 단어의 등장 횟수
    selected_words = np.where(words_cnt >= 50)[0]
    remain = words[:, selected_words]
    return remain, selected_words

x_train_BoW_reduced, indices = reduce_dimension(x_train_BoW)

print("원본 BoW 크기:", x_train_BoW.shape)
print("차원 축소 후 크기:", x_train_BoW_reduced.shape)

- 분류

In [ ]:
# TEST 데이터를 전처리
x_test = test_data["review"]
x_test_preprocessed = [apply_regex(pattern, str(x[1])) for x in tqdm(x_test.iteritems(), total=len(x_test), desc="pre-processing data")]
x_test_tokenized = [tokenize_words(x) for x in tqdm(x_test_preprocessed, desc="tokenizing data")]
x_test_stopwords_excluded = [exclude_stopwords(x) for x in x_test_tokenized]
x_test_embedded = [embed_tokens(x, mode="TEST") for x in tqdm(x_test_stopwords_excluded, desc="embedding data")]
M = embedding_value
x_test_BoW = to_BoW_representation(x_test_embedded)
x_test_BoW_reduced = x_test_BoW[:, indices]

In [ ]:
# TEST 데이터에 대한 예측 수행 코드 작성
from lightgbm import LGBMClassifier
clf = LGBMClassifier(n_estimators=3000)

clf.fit(x_train_BoW_reduced, y_train)

In [ ]:
pred = clf.predict(x_test_BoW_reduced)
print(sum(pred == y_train) / len(y_train))

submit = pd.read_csv("sample_submission.csv", index_col=0)
# TEST 데이터에 대한 예측 값을 csv로 저장하는 코드 작성
submit['rating'] = pred
submit.to_csv("submit_logi.csv")